# 图形标注
***


虽然本节出现在连接器章内，本话题适用于与一般的模型定义相关的图形标注。因此，这里提供的信息可以对Modelica的许多方面提供有益的参考。


## 图形层

有两种不同的表现形式可以去描述Modelica的实体的外观。一个是所谓“图标”表示。另一个则是所谓“简图”表示。在Modelica里，图标表示展现的是从“外面”观察模型时看到的内容。一般来说，图标包括一些独特的视觉表现形式。此外，通过替换（稍后我们会进行介绍），它也包含了关于该实体的进一步信息。


在另一方面，“简图”表示是用来展现组件的“内部”视图。简图表示通常用于Modelica组件图形化的详细文档。特别是当这些内容对于“图标”视图来说过于仔细的时候。


一个定义在“图标”层所展现的外观是由Icon标注指定的（在之前对图形连接器讨论里有简要介绍）。毫无疑问，定义在“简图”层所展现的外观则是由Diagram标注指定的。两者均是直接出现在定义里，不和声明或者extends条款等现有元素相关联。


一般来说，大部分的定义都包括一个“图标”的表示，但只有少数有空去包括“简图”表示。不过，事实上尽管是在不同的情况下进行的渲染，图形表示的规范却是相同的。


>## 使用Icon的例子

>>在这本书的其余部分，我们将利用Icon标注举例介绍图形标注。这些例子对Diagram标注同样有效。但由于Icon标注相对常见，关于图形标注的进一步例子仅会出现Icon标注里。


## 通用图形定义

下面的定义将在本节中引用：


```Modelica
type DrawingUnit = Real(final unit="mm");
type Point = DrawingUnit[2] "{x, y}";
type Extent = Point[2]
  "Defines a rectangular area {{x1, y1}, {x2, y2}}";
type Color = Integer[3](min=0, max=255) "RGB representation";
constant Color Black = zeros(3);
type LinePattern = enumeration(None, Solid, Dash, Dot, DashDot, DashDotDot);
type FillPattern = enumeration(None, Solid, Horizontal, Vertical,
                               Cross, Forward, Backward,
                               CrossDiag, HorizontalCylinder,
                               VerticalCylinder, Sphere);
type BorderPattern = enumeration(None, Raised, Sunken, Engraved);
type Smooth = enumeration(None, Bezier);
type Arrow = enumeration(None, Open, Filled, Half);
type TextStyle = enumeration(Bold, Italic, UnderLine);
type TextAlignment = enumeration(Left, Center, Right);

record FilledShape "Style attributes for filled shapes"
  Color lineColor = Black "Color of border line";
  Color fillColor = Black "Interior fill color";
  LinePattern pattern = LinePattern.Solid "Border line pattern";
  FillPattern fillPattern = FillPattern.None "Interior fill pattern";
  DrawingUnit lineThickness = 0.25 "Line thickness";
end FilledShape;
```


另外，很多我们马上会讨论到的标注包括一组在以下record定义里的共同元素：


```Modelica
partial record GraphicItem
  Boolean visible = true;
  Point origin = {0, 0};
  Real rotation(quantity="angle", unit="deg")=0;
end GraphicItem;
```


为了明晰地介绍表示图形元素的常用标注，我们会以GraphicItem为基础扩展出其他模型。


## Icon和Diagram标注


以下数据描述了应该出现在模型图标层的元素：


```Modelica
record Icon "Representation of the icon layer"
  CoordinateSystem coordinateSystem(extent = {{-100, -100}, {100, 100}});
  GraphicItem[:] graphics;
end Icon;
```


其中坐标系统中的数据被定义为：


```Modelica
record CoordinateSystem
  Extent extent;
  Boolean preserveAspectRatio=true;
  Real initialScale = 0.1;
  DrawingUnit grid[2];
end CoordinateSystem;
```


换言之，该Icon标注包括了coordinateSystem定义内的坐标系信息。而且，标注还包括了graphics内的图形物件数组。


Diagram标注的定义也是一样：


```Modelica
record Diagram "Representation of the diagram layer"
  CoordinateSystem coordinateSystem(extent = {{-100, -100}, {100, 100}});
  GraphicItem[:] graphics;
end Diagram;
```


## 图形物件

Modelica标准里有许多不同的图形物件可以放入Icon或Diagram标注内的graphics矢量。其定义在这里列出以供参考。


### Line


```Modelica
record Line
  extends GraphicItem;
  Point points[:];
  Color color = Black;
  LinePattern pattern = LinePattern.Solid;
  DrawingUnit thickness = 0.25;
  Arrow arrow[2] = {Arrow.None, Arrow.None} "{start arrow, end arrow}";
  DrawingUnit arrowSize=3;
  Smooth smooth = Smooth.None "Spline";
end Line;
```


### Polygon


```Modelica
record Polygon
  extends GraphicItem;
  extends FilledShape;
  Point points[:];
  Smooth smooth = Smooth.None "Spline outline";
end Polygon;
```


### Rectangle


```Modelica
record Rectangle
  extends GraphicItem;
  extends FilledShape;
  BorderPattern borderPattern = BorderPattern.None;
  Extent extent;
  DrawingUnit radius = 0 "Corner radius";
end Rectangle;
```


### Ellipse


```Modelica
record Ellipse
  extends GraphicItem;
  extends FilledShape;
  Extent extent;
  Real startAngle(quantity="angle", unit="deg")=0;
  Real endAngle(quantity="angle", unit="deg")=360;
end Ellipse;
```


### Text


```Modelica
record Text
  extends GraphicItem;
  extends FilledShape;
  Extent extent;
  String textString;
  Real fontSize = 0 "unit pt";
  String fontName;
  TextStyle textStyle[:];
  Color textColor=lineColor;
  TextAlignment horizontalAlignment = TextAlignment.Center;
end Text;
```



### Bitmap


```Modelica
record Bitmap
  extends GraphicItem;
  Extent extent;
  String fileName "Name of bitmap file";
  String imageSource "Base64 representation of bitmap";
end Bitmap;
```


## 继承图形标注

当一个模型定义从另一个定义继承时，图形标注在默认情况下也会继承。不过，这种行为可以通过对extends子句添加以下标注来控制（分别对应图标层和简图层）：


```Modelica
record IconMap
  Extent extent = {{0, 0}, {0, 0}};
  Boolean primitivesVisible = true;
end IconMap;

record DiagramMap
  Extent extent = {{0, 0}, {0, 0}};
  Boolean primitivesVisible = true;
end DiagramMap;
```


在这两种情况下， extent数据允许继承图形元素的位置进行调整。将primitivesVisible设置为false则会隐藏所继承的图形元素。


## 替换

Text标注内的textString字段可以包含替换模式。标注支持以下的替换模式：

- %name － 这个字符串模式会被给定定义的实例名称所取代。
- %class － 这个字符串模式会被这个定义的名称所取代。

- %$<name>$，其中$<name>$为参数名称 - 这个模式会被替换为相应参数的值。

- %% － 这个模式会被替换为%。


## 组合以上内容

在讨论了图形标注的这些方面后，让我们回顾在对图形连接器的讨论中提出的图标定义。


```Modelica
      Icon(graphics={
          Ellipse(
            extent={{-100,100},{100,-100}},
            lineColor={0,0,255},
            fillColor={85,170,255},
            fillPattern=FillPattern.Solid),
          Rectangle(
            extent={{-10,58},{10,-62}},
            fillColor={0,128,255},
            fillPattern=FillPattern.Solid,
            pattern=LinePattern.None),
          Rectangle(
            extent={{-60,10},{60,-10}},
            fillColor={0,128,255},
            fillPattern=FillPattern.Solid,
            pattern=LinePattern.None,
            lineColor={0,0,0}),
          Text(
            extent={{-100,-100},{100,-140}},
            lineColor={0,0,255},
            fillColor={85,170,255},
            fillPattern=FillPattern.Solid,
            textString="%name")}),
```


这里我们看到了与PositivePin定义相关联的annotation是个模型标注。此外，我们可以看到这个标注内Icon元素包含一系列的图形物体。第一个图形物体是Ellipse标注。紧接着的是两个Rectangle标注。最后是Text（这里使用了前面所讨论的替换）。


注意数据在annotation里的呈现方式与我们在前面讨论过的记录（record）定义中描述的数据一致。